## Dense Layer sampling

Analysis of order and resolutions of hidden layers in UNet model

In [44]:
%load_ext autoreload
%autoreload 2

In [45]:
import sys
sys.path.append('../')

# local imports
from setup import setup_device, setup_datamodule, load_model

device = setup_device('cuda:2')
print(device, type(device))

model = load_model('../pre-trained/trained_UNets/mnmv2-00-02_22-11-2024-v1.ckpt', device)

cuda:2 <class 'torch.device'>
Loaded as Lightning module


In [46]:
datamodule = setup_datamodule()
data_sample = next(iter(datamodule.val_dataloader()))['input']

data_sample.size() # [batch size, channels, width, hegiht]

torch.Size([32, 1, 256, 256])

In [47]:
from torchinfo import summary

summary(model, input_size=data_sample.size())

Layer (type:depth-idx)                                                                Output Shape              Param #
LightningSegmentationModel                                                            [32, 4, 256, 256]         --
├─UNet: 1-1                                                                           [32, 4, 256, 256]         --
│    └─Sequential: 2-1                                                                [32, 4, 256, 256]         --
│    │    └─ResidualUnit: 3-1                                                         [32, 16, 128, 128]        7,284
│    │    └─SkipConnection: 3-2                                                       [32, 32, 128, 128]        785,872
│    │    └─Sequential: 3-3                                                           [32, 4, 256, 256]         1,305
Total params: 794,461
Trainable params: 794,461
Non-trainable params: 0
Total mult-adds (G): 47.97
Input size (MB): 8.39
Forward/backward pass size (MB): 1962.93
Params size (MB):

In [48]:
from adapters.main import capture_convolution_layers

wrapper, layer_names = capture_convolution_layers(model, device, data_sample)

In [49]:
for count, (module, output) in enumerate(wrapper.layer_activations.items()):
    print(f"{count+1:0>3}. Layer: {module}, Output Shape: {output.shape}")

001. Layer: model.model.0.conv, Output Shape: torch.Size([32, 1, 256, 256])
002. Layer: model.model.0.conv.unit0, Output Shape: torch.Size([32, 1, 256, 256])
003. Layer: model.model.0.conv.unit0.conv, Output Shape: torch.Size([32, 1, 256, 256])
004. Layer: model.model.0.conv.unit0.adn, Output Shape: torch.Size([32, 16, 128, 128])
005. Layer: model.model.0.conv.unit0.adn.N, Output Shape: torch.Size([32, 16, 128, 128])
006. Layer: model.model.0.conv.unit0.adn.D, Output Shape: torch.Size([32, 16, 128, 128])
007. Layer: model.model.0.conv.unit0.adn.A, Output Shape: torch.Size([32, 16, 128, 128])
008. Layer: model.model.0.conv.unit1, Output Shape: torch.Size([32, 16, 128, 128])
009. Layer: model.model.0.conv.unit1.conv, Output Shape: torch.Size([32, 16, 128, 128])
010. Layer: model.model.0.conv.unit1.adn, Output Shape: torch.Size([32, 16, 128, 128])
011. Layer: model.model.0.conv.unit1.adn.N, Output Shape: torch.Size([32, 16, 128, 128])
012. Layer: model.model.0.conv.unit1.adn.D, Output Sha

Some convolutional blocks are compound consist of subsequent units with normalization and dropouts. Let us do some filtering for compound blocks and choose the final output of those units:

In [50]:
import re

unit_pattern = r"(unit\d+)"

dense_layer_keys = []

last_unit_layer = ""
for count, conv_name in enumerate(wrapper.layer_activations.keys()):
    match = re.search(unit_pattern, conv_name)
    
    if match:
        unit_name = match.group(1)

        if not last_unit_layer == "":
            if not re.search(unit_pattern, last_unit_layer).group(1) == unit_name:
                # we came to next unit, append the last layer from previously viewed
                dense_layer_keys.append(last_unit_layer)
        last_unit_layer = conv_name

    else:
        # it is not a part of unit with normalizations and dropouts, so we can simply append
        dense_layer_keys.append(conv_name)

print(f"Sampled {len(dense_layer_keys)} convolutional layers within the model:")

Sampled 26 convolutional layers within the model:


In [51]:
dense_layer_keys

['model.model.0.conv',
 'model.model.0.conv.unit0.adn.A',
 'model.model.0.conv.unit1.adn.A',
 'model.model.0.conv.unit2.adn.A',
 'model.model.1.submodule.0.conv',
 'model.model.0.conv.unit3.adn.A',
 'model.model.1.submodule.0.conv.unit0.adn.A',
 'model.model.1.submodule.0.conv.unit1.adn.A',
 'model.model.1.submodule.0.conv.unit2.adn.A',
 'model.model.1.submodule.1.submodule.0.conv',
 'model.model.1.submodule.0.conv.unit3.adn.A',
 'model.model.1.submodule.1.submodule.0.conv.unit0.adn.A',
 'model.model.1.submodule.1.submodule.0.conv.unit1.adn.A',
 'model.model.1.submodule.1.submodule.0.conv.unit2.adn.A',
 'model.model.1.submodule.1.submodule.1.submodule.conv',
 'model.model.1.submodule.1.submodule.0.conv.unit3.adn.A',
 'model.model.1.submodule.1.submodule.1.submodule.conv.unit0.adn.A',
 'model.model.1.submodule.1.submodule.1.submodule.conv.unit1.adn.A',
 'model.model.1.submodule.1.submodule.1.submodule.conv.unit2.adn.A',
 'model.model.1.submodule.1.submodule.2.0.conv',
 'model.model.1.su

Let us group by resolution and see how many layers for each of the resolutions:

In [52]:
size_by_layers = {}

for layer_key in dense_layer_keys:
    size = wrapper.layer_activations[layer_key].shape
    size_str = ', '.join(map(str, size))
    size_id = f'[{size_str}]'
    if size_id in size_by_layers:
        size_by_layers[size_id].append(layer_key)
    else:
        size_by_layers[size_id] = [layer_key]

print(f'Found {len(size_by_layers.keys())} different resolutions:')
for count, (key, items) in enumerate(size_by_layers.items()):
    print(f"{count+1}. {len(items)} layer(s) for resolution {key}")


Found 9 different resolutions:
1. 1 layer(s) for resolution [32, 1, 256, 256]
2. 6 layer(s) for resolution [32, 16, 128, 128]
3. 6 layer(s) for resolution [32, 32, 64, 64]
4. 5 layer(s) for resolution [32, 64, 32, 32]
5. 4 layer(s) for resolution [32, 128, 32, 32]
6. 1 layer(s) for resolution [32, 192, 32, 32]
7. 1 layer(s) for resolution [32, 64, 64, 64]
8. 1 layer(s) for resolution [32, 32, 128, 128]
9. 1 layer(s) for resolution [32, 4, 256, 256]


In [53]:
size_by_layers

{'[32, 1, 256, 256]': ['model.model.0.conv'],
 '[32, 16, 128, 128]': ['model.model.0.conv.unit0.adn.A',
  'model.model.0.conv.unit1.adn.A',
  'model.model.0.conv.unit2.adn.A',
  'model.model.1.submodule.0.conv',
  'model.model.0.conv.unit3.adn.A',
  'model.model.1.submodule.2.1.conv'],
 '[32, 32, 64, 64]': ['model.model.1.submodule.0.conv.unit0.adn.A',
  'model.model.1.submodule.0.conv.unit1.adn.A',
  'model.model.1.submodule.0.conv.unit2.adn.A',
  'model.model.1.submodule.1.submodule.0.conv',
  'model.model.1.submodule.0.conv.unit3.adn.A',
  'model.model.1.submodule.1.submodule.2.1.conv'],
 '[32, 64, 32, 32]': ['model.model.1.submodule.1.submodule.0.conv.unit0.adn.A',
  'model.model.1.submodule.1.submodule.0.conv.unit1.adn.A',
  'model.model.1.submodule.1.submodule.0.conv.unit2.adn.A',
  'model.model.1.submodule.1.submodule.1.submodule.conv',
  'model.model.1.submodule.1.submodule.0.conv.unit3.adn.A'],
 '[32, 128, 32, 32]': ['model.model.1.submodule.1.submodule.1.submodule.conv.unit0.